In [ ]:
import os
import pathlib
import sys

import pandas as pd

pd.options.mode.copy_on_write = True
pd.options.display.max_columns = None

from transit import vmgo

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
data_path = pathlib.Path(os.path.abspath("../data"))
data_path

In [ ]:
output_path = pathlib.Path(os.path.abspath("../output/roanoke"))
output_path.mkdir(exist_ok=True, parents=True)
output_path

In [ ]:
plot_path = pathlib.Path(output_path / 'plots')
plot_path.mkdir(exist_ok=True)
plot_path

In [ ]:
df_alighting = pd.read_csv(
    str(data_path / 'roanoke' / 'Alighting-May24-Apr25.csv'),
    header=[2],
    low_memory=False,
    skip_blank_lines=True,
)
df_alighting

In [ ]:
df_boarding = pd.read_csv(
    str(data_path / 'roanoke' / 'Boarding-May24-Apr25.csv'),
    header=[2],
    low_memory=False,
    skip_blank_lines=True,
)
df_boarding

In [ ]:
df_ridership = (
    df_alighting.set_index('Stop Name')
    .join(
        df_boarding.set_index('Stop Name'),
        on='Stop Name',
        how='outer'
    )
    .reset_index()
)
df_ridership['Stop Name'] = df_ridership['Stop Name'].str.replace(' (no number)', '')
df_ridership

In [ ]:
regions = vmgo.load_from_datadir(data_path / 'vmgo')
regions

In [ ]:
# create dataframe rows
rows = []

for region in regions.values():
    for route in region.routes:
        for directions in route.directions:
            for stop in directions.stops:
                rows.append({
                    "region": region.name,
                    "route": route.name,
                    "direction": directions.name,
                    "stop": stop.name,
                    "lat": stop.lat,
                    "lon": stop.lon,
                })
df_routes = pd.DataFrame(rows)
df_routes

In [ ]:
df = df_routes.groupby(['stop']).count()
df['route_count'] = df['direction']
df = df.reset_index()[['stop', 'route_count']]
df_stop_route_count = df
df_stop_route_count

In [ ]:
df = df_routes.merge(
    df_ridership.rename(
        columns={
            "Stop Name": "stop",
            "Sum of Passenger Alightings": "alightings",
            "Sum of Passenger Boardings": "boardings",
        }
    ),
    on='stop',
    how='left',
).merge(
    df_stop_route_count,
    on='stop',
    how='left',
)
df['alightings'] = df['alightings'].str.replace(',', '').astype(float)
df['boardings'] = df['boardings'].str.replace(',', '').astype(float)
df = df.fillna(0)
df_route_ridership = df
df_route_ridership

In [ ]:
ridership_path = output_path / 'ridership'
ridership_path.mkdir(exist_ok=True)

df_route_ridership.sort_values('boardings', ascending=False).to_csv(
    ridership_path / 'ridership-with-stops-sorted-boarding.csv')
df_route_ridership.sort_values('alightings', ascending=False).to_csv(
    ridership_path / 'ridership-with-stops-sorted-alighting.csv')

In [ ]:
df_stop_lat_lons = df_route_ridership[['stop', 'lat', 'lon']].drop_duplicates()
df_stop_lat_lons

In [ ]:
df = df_stop_lat_lons.merge(
    df_ridership.rename(
        columns={
            "Stop Name": "stop",
            "Sum of Passenger Alightings": "alightings",
            "Sum of Passenger Boardings": "boardings",
        }
    ),
    on='stop',
    how='left',
)
df['alightings'] = df['alightings'].str.replace(',', '').astype(float)
df['boardings'] = df['boardings'].str.replace(',', '').astype(float)
#df = df.fillna(0)
df_stop_ridership = df
df_stop_ridership

In [ ]:
import ipyleaflet.basemaps
import ipywidgets

pymap = ipyleaflet.Map(
    center=(df_routes['lat'].mean(), df_routes['lon'].mean()), zoom=12, min_zoom=1, max_zoom=20,
    layout=ipywidgets.Layout(width='auto', min_height='400px')
)

markers = []
for index, row in df_stop_ridership.sort_values('alightings', ascending=False)[:40].iterrows():
    html = ''.join([
        '<span style="color:#000; font-size:8pt;">',
        row['stop'],
        '<br/>',
        f"{int(row['boardings'])} boarded / {int(row['alightings'])} alighted",
        '</span',
    ])
    icon = ipyleaflet.DivIcon(html=html, bg_pos=[0, 0], icon_size=[100, 70])
    marker = ipyleaflet.Marker(
        location=(row['lat'], row['lon']),
        title=row['stop'],
        icon=icon,
        draggable=False,
    )
    markers.append(marker)

marker_cluster = ipyleaflet.MarkerCluster(markers=markers)
pymap.add(marker_cluster)
pymap.save(output_path / 'ridership' / 'map-top-40-alightings.html', title='Top 40 Stops by Alighting Numbers')
pymap

In [ ]:
import matplotlib.pyplot as plt
import shutil

In [ ]:
routes_and_directions = df_route_ridership[['route', 'direction']].drop_duplicates()

In [ ]:
plot_dir = plot_path / 'ridership-full-raw'
shutil.rmtree(plot_dir, ignore_errors=True)
plot_dir.mkdir(exist_ok=True)

for index, row in routes_and_directions.iterrows():
    df = df_route_ridership
    df = df[(df['route'] == row['route']) & (df['direction'] == row['direction'])]

    fig, ax = plt.subplots(layout='constrained')
    fig.set_size_inches(12, 8)

    ax.set_title('\n'.join([
        'Roanoke, VA - Boarding/Alighting: May 1, 2024 - April 30, 2025',
        row['route'],
        row['direction'],
        '(some stops are shared with other routes)',
    ]))
        
    ax.bar(df['stop'], df['boardings'] / df['route_count'], color='blue', label='boardings')
    ax.bar(df['stop'], -df['alightings'] / df['route_count'], color='red', label='alightings')
    ax.legend(loc="best")

    ax.tick_params(axis='x', labelrotation=90)

    # plt.show()
    plt.savefig(plot_dir / f'{row["route"].strip().replace('/', ' or ')} --- {row["direction"].strip().replace('/', ' or ')}.png')
    plt.close()

In [ ]:
plot_dir = plot_path / 'ridership-full-adjusted'
shutil.rmtree(plot_dir, ignore_errors=True)
plot_dir.mkdir(exist_ok=True)

for index, row in routes_and_directions.iterrows():
    df = df_route_ridership
    df = df[(df['route'] == row['route']) & (df['direction'] == row['direction'])]

    fig, ax = plt.subplots(layout='constrained')
    fig.set_size_inches(12, 8)

    ax.set_title('\n'.join([
        'Roanoke, VA - Adjusted Boarding/Alighting: May 1, 2024 - April 30, 2025',
        row['route'],
        row['direction'],
        '(with total ridership of shared stops divided by number of shared routes)',
    ]))
        
    ax.bar(df['stop'], df['boardings'] / df['route_count'], color='blue', label='boardings')
    ax.bar(df['stop'], -df['alightings'] / df['route_count'], color='red', label='alightings')
    ax.legend(loc="best")

    ax.tick_params(axis='x', labelrotation=90)

    # plt.show()
    plt.savefig(plot_dir / f'{row["route"].strip().replace('/', ' or ')} --- {row["direction"].strip().replace('/', ' or ')}.png')
    plt.close()

In [ ]:
plot_dir = plot_path / 'ridership-less-than-10000'
shutil.rmtree(plot_dir, ignore_errors=True)
plot_dir.mkdir(exist_ok=True)

for index, row in routes_and_directions.iterrows():
    df = df_route_ridership
    df = df[(df['route'] == row['route']) & (df['direction'] == row['direction'])]
    df = df[df['boardings'] < 10000]
    df = df[df['alightings'] < 10000]

    fig, ax = plt.subplots(layout='constrained')
    fig.set_size_inches(12, 6)

    ax.set_title('Roanoke, VA Boarding/Alighting: May 1, 2024 - April 30, 2025\n' + row['route'] + '\n' + row[
        'direction'] + '\n(Stops with < 10,0000 boarders/alighters)')
    ax.bar(df['stop'], df['boardings'], color='blue', label='boardings')
    ax.bar(df['stop'], -df['alightings'], color='red', label='alightings')
    ax.legend(loc="best")

    ax.tick_params(axis='x', labelrotation=90)

    #plt.show()
    plt.savefig(plot_dir / f'{row["route"].strip().replace('/', ' or ')} --- {row["direction"].strip().replace('/', ' or ')}.png')
    plt.close()